In [35]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from visualisation import (
    create_research_landscape_treemap, 
    analyze_keyword_trends,
    create_keyword_trends_visualization,
    create_treemap_data
)
from utils import load_categories, load_classification_results
from config import FIGURES_DIR, EXTRACTED_KEYWORDS_PATH, EXTRACTED_KEYWORDS_DIR
from inspect_ai.log import read_eval_log
from inspect_ai import eval_retry

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
fig = create_research_landscape_treemap(max_keywords_per_category=10, max_categories_per_for=5, max_for_classes=5)
fig.write_html(FIGURES_DIR / "research_landscape_treemap.html")

In [44]:
analysis = analyze_keyword_trends(keywords_path=EXTRACTED_KEYWORDS_PATH)
fig = create_keyword_trends_visualization(analysis)
fig.write_html(FIGURES_DIR / "keyword_trends.html")
